<a href="https://colab.research.google.com/github/JanEggers-hr/GPT-Zufalls-Experiment/blob/main/GPT_Antwort_auf_R%C3%A4tselfrage_testen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wie oft löst GPT das Rätsel?

Ein Skript, das GPT ein Rätsel stellt - und dann mit einer weiteren GPT-Anfrage versucht festzustellen, wie häufig die Antwort korrekt ist.

Das Experiment soll zeigen, welche Rolle der Zufall in der Antwort des Sprachmodells spielt. Mehr dazu in meinem Blog: https://janeggers.tech/ - Version 1a vom 9.1.2024

**Benötigt wird ein API-Token von OpenAI.**

Unten beim Programmcode auf den Play-Button drücken, dann warten - und wenn alle Bibliotheken geladen sind, den Code einkopieren und Return drücken.

Das Programm spricht die OpenAI-Server an - Wenn der API-Key akzeptiert wird, kommt eine entsprechende Meldung.

In [ ]:
#@title Vorbereitung
import requests
import json
import math
import markdown

# ipywidgets ist schon installiert
import ipywidgets as widgets
from IPython.display import display

# Modelle und Kosten definieren
# Kosten in US-Dollar je 1000 Tokens
# Könnte die Modelle auch über die API holen, aber so kann ich die Kosten
# mitgeben. Entsprechend OpenAI-Preisliste Dezember 2023. https://openai.com/pricing

models_token_info = {
          'gpt-4-1106-preview': {
                                        'output_price': 0.03,
                                        'input_price': 0.01,
                                        'max_tokens': 128000
                                      },
          'gpt-3.5-turbo-1106': {
                                        'output_price': 0.002,
                                        'input_price': 0.001,
                                        'max_tokens': 16385
                                      },
          'gpt-4': {
                                        'pricing': 0.03,
                                        'input_price': 0.01,
                                        'max_tokens': 8192
                                      },
          'gpt-3.5-turbo-0613': {
                                        'output_price': 0.002,
                                        'input_price': 0.001,
                                        'max_tokens': 4096
                                      }}


# Tokenizer Tiktoken einbinden
!pip install -q tiktoken
import tiktoken
print("Tokenizer tiktoken geladen.")

# OpenAI-API-Library einbinden
!pip install -q openai
from openai import OpenAI
print("OpenAI-API-Library geladen.")

##### Der eigentliche Code! #####

from getpass import getpass
from google.colab import userdata

key_needed = True
try:
  ai_client = OpenAI(api_key = userdata.get('openai'))
except:
  print("OpenAI-Key benötigt")
else:
  print("*** API-Key gültig! ***")
  key_needed = False

while key_needed:
    try:
        # Testweise Modelle abfragen
        ai_client = OpenAI(api_key = getpass("OpenAI-API-Key eingeben: "))
        models = ai_client.models.list()
        # Returns a list of model objects
        # Erfolg?
        print()
        print("*** API-Key gültig! ***")
        key_needed = False
    except Exception as e:
        print("Fehler bei Abfrage; ist der API-Key möglicherweise ungültig?", e)

spent_tokens = 0        # Wie viele Tokens wurden bisher über die API abgefragt?
spent_dollars = 0.00    # Zu welchem Preis?


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
Tokenizer tiktoken geladen.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<

# API-Key gültig?

Jetzt auf den nächsten "Play"-Button klicken - die Einstell-Elemente werden erzeugt und angezeigt. Temperatur, KI-Modell, Fragetext und Anzahl der Durchgänge anpassen, wenn gewünscht - dann "Starten" klicken, das Experiment beginnt:

* Die Rätselfrage wird der KI gestellt, so oft hintereinander wie eingestellt.
* Eine weitere Anfrage an die KI bewertet, wie gut die Antworten waren.
* Am Ende werden die statistischen Abweichungen bei den genannten Zahlen der Brüder und Schwestern berechnet...
* ...und die drei häufigsten Antwort-Anfänge aufgelistet (hierbei werden nur die ersten 50 Buchstaben verglichen).

Eine Tabelle der Antworten wird als Excel_Datei erzeugt und heruntergeladen.

Das Rätsel stammt aus dem [LLMonitor-Benchmark](https://benchmarks.llmonitor.com/) von [@vincelwt](https://vincelwt.com); wer will, kann es gern anpassen.

In [ ]:
#@title Code für das Experiment

# ipywidgets ist schon installiert
import ipywidgets as widgets
import requests
from IPython.display import display
from google.colab import files
import pandas as pd
import json
import math
import markdown

# Modelle und Kosten definieren
# Kosten in US-Dollar je 1000 Tokens
models_token_info = {
          'gpt-3.5-turbo-1106': {
                                        'output_price': 0.002,
                                        'input_price': 0.001,
                                        'max_tokens': 16385
                                      },
          'gpt-4-1106-preview': {
                                        'output_price': 0.03,
                                        'input_price': 0.01,
                                        'max_tokens': 128000
                                      },
          'gpt-3.5-turbo': {
                                        'output_price': 0.002,
                                        'input_price': 0.001,
                                        'max_tokens': 4096
                                      },
}

##### Der eigentliche Code! #####

spent_tokens = 0        # Wie viele Tokens wurden bisher über die API abgefragt?
spent_dollars = 0.00    # Zu welchem Preis?

# Define the HTML and JavaScript code for the spinner animation
spinner_html = """
<div class="loader"></div>
<style>
.loader {
  border: 8px solid #f3f3f3;
  border-top: 8px solid #3498db;
  border-radius: 50%;
  width: 25px;
  height: 25px;
  animation: spin 2s linear infinite;
  margin: 20px auto;
}

@keyframes spin {
  0% { transform: rotate(0deg); }
  100% { transform: rotate(360deg); }
}
</style>
"""

# Hilfsfunktion: Token berechnen
def calculate_tokens(string: str) -> int:
    """Returns the number of tokens in a text string."""
    # cl100k_base ist der Tokenizer für Davinci, GPT-3 und GPT-4
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

def output_pricing(tokens):
    price = models_token_info.get(model)['output_price']
    # Kosten in Dollar zurückgeben
    return(tokens * price / 1000)

def input_pricing(tokens):
    price = models_token_info.get(model)['input_price']
    # Kosten in Dollar zurückgeben
    return(tokens * price / 1000)

def gpt(g_system,g_previous_messages,g_prompt,g_model,temperature = 0,json=False):
    # Systemprompt und few-shots zusammenbinden
    #
    global max_tokens
    global stoptokens
    global spent_tokens
    global spent_dollars
    if json==True:
        m_type = {"type": "json_object"}
    else:
        m_type = {"type": "text"}

    prompts = [
            {"role": "system", "content": g_system},
            *g_previous_messages,
            {"role": "user", "content": g_prompt},
        ]
    response = ai_client.chat.completions.create(
        messages=prompts,
        model=g_model,
        max_tokens=max_tokens,
        n=1,
        response_format= m_type,
        stream = False,
        temperature=temperature,
    )
    # Anzahl verbrauchtert Tokens anpassen
    spent_tokens += response.usage.total_tokens
    spent_dollars += output_pricing(response.usage.completion_tokens) + input_pricing(response.usage.prompt_tokens)
    token_usage_text = f'<b>Verbrauchte Token:</b> {spent_tokens} ($ {spent_dollars:.3f}) '
    text_tokens.value = token_usage_text
    return(response.choices[0].message.content)

# Funktion wird bei Veränderung ausgeführt
def update_params(change):
    global temperature
    global max_tokens
    global system_prompt
    global model
    global stoptokens
    global best_of

# Vorbereitungen für die Einstellungen sind getan - jetzt die OpenAI-Libraries
print("Widgets eingerichtet.")

import tiktoken
from openai import OpenAI

max_tokens = 256
stoptokens = "###"

def beantworte(temp):
    global raetsel_prompt
    g_system = ''
    g_prompt = raetsel_prompt
    previous_messages = []
    t = gpt(g_system,
            previous_messages,
            g_prompt,
            g_model = dropdown_model.value",
            temperature = temp)
    print("*",end="")
    return(t)


beurteilung_prompt = '''
Du bist Datenanalyst. Du liest Antworten auf eine Quizfrage nach der
Anzahl der Brüder und Schwestern von Anna. Dabei ist Anna eine der Schwestern.
Lies den Antwort-Text und gib die Anzahl der Brüder und Schwestern zurück, die
genannt sind. Produziere eine JSON-Datei. Nutze diese Form:
{ "B":1, "S":2 }
'''
beurteilung_beispiele = [{"role": "user", "content": "Das Rätsel kann auf den ersten Blick verwirrend erscheinen, aber die Lösung ist eigentlich recht einfach. Wenn Anna drei Brüder hat, dann ist sie selbst eine Schwester für jeden ihrer Brüder. Das bedeutet, dass Anna und ihre drei Brüder insgesamt vier Geschwister sind."},
                      {"role": "assistant", "content": '{"B":3, "S":1}'},
                      {"role": "user", "content": "Vier Brüder und Anna und ihre zwei Schwestern."},
                      {"role": "assistant", "content": '{"B":4, "S":3}'}]


def beurteile(text):
    global beurteilung_prompt
    global beurteilung_beispiele
    global html_spinner
    g_system = beurteilung_prompt
    previous_messages = beurteilung_beispiele
    t = gpt(g_system,
            previous_messages,
            text,
            g_model = "gpt-3.5-turbo-1106",
            temperature = 0,
            json = True)
    print("+",end="")
    # Gib das JSON als dict zurück
    return(t)


# Slider für die Temperatur (Default: 1)
slider_temperatur = widgets.FloatSlider(
    value=1.0,
    min=0,
    max=2.0,
    step=0.1,
    description='Temperatur:',
    orientation='horizontal',
    readout=True
)


slider_samples = widgets.IntSlider(
    value=100,
    min=10,
    max=500,
    step = 10,
    description='Durchgänge:',
    orientation='horizontal',
    readout=True,
    layout=widgets.Layout(width='50%')
)

textbox_max_tokens = widgets.Text(
    value='256',
    placeholder='0',
    description='Max. Antwort:',
)

text_tokens = widgets.HTML(
    value = '<b>Verbrauchte Token</b>: 0 ($0.00)'
)

dropdown_model = widgets.Dropdown(
    # Nimm die oben definierte Preisliste als Basis
    options=list(models_token_info.keys()),
    value=list(models_token_info.keys())[0],
    description='Modell:',
)

g_prompt = '''Ein Rätsel: Anna hat drei Brüder, jeder hat zwei Schwestern.
Wie viele Brüder und Schwestern sind es insgesamt?'''

area_raetsel = widgets.Textarea(
    value = g_prompt,
    rows=10,
    description = 'Rätselfrage:',
    layout=widgets.Layout(width='80%')
)

html_samples_tokens = widgets.HTML(
    value = f'<b>Samples in Token</b>: {spent_tokens} '
)

button_start_experiment = widgets.Button(
    description='Starten',
    layout=widgets.Layout(width='15%'),
)

# Die Zutaten sind da, jetzt bereite sie zu!

def my_main(change):
  global temperatur
  global num_samples
  display(text_tokens)
  print(f"Sammele {num_samples} Antworten")
  html_spinner = widgets.HTML(spinner_html)
  display(html_spinner)
  antworten = [beantworte(temperatur) for _ in range(num_samples)]
  df = pd.DataFrame({
      'Text': antworten
      })
  html_spinner.close()
  print()
  print(f"Beurteile die {num_samples} Antworten")
  # GPT3.5 ein Urteil fällen lassen
  html_spinner = widgets.HTML(spinner_html)
  display(html_spinner)
  bewertungen = (df['Text'].apply(beurteile))
  html_spinner.close()
  df['BS'] = bewertungen.apply(json.loads)

  # GPT gibt ein JSON zurück; Rückgabewert der beurteilen() Funktion ist ein dict
  # Spalte es in zwei Einzelwerte auf
  df['B'] = df['BS'].apply(lambda x: x['B'])
  df['S'] = df['BS'].apply(lambda x: x['S'])

  # dict-Spalte rausschmeißen, neue Spalte erzeugen, die Summe erzeugt und prüft
  df.drop('BS', axis=1, inplace=True)
  df['G'] = df['B'] + df['S']
  df['OK'] = df.apply(lambda row: row['B'] == 3 and row['S'] == 2, axis=1)

  fname = f"./raetselantworten_{dropdown_model.value}_{temperatur:.1f}.xlsx"
  df.to_excel(fname, index=False)
  # Datei herunterladen
  files.download(fname)

  # Auswertung
  print()
  print(f"--- AUSWERTUNG FÜR {dropdown_model.vale} mit Temperatur {temperatur:.1f}---")
  prozent_richtig = df['OK'].sum() / num_samples
  print(f"Richtig beantwortet: {prozent_richtig * 100}%")
  print("Brüder: ")
  print(f" Median: {df['B'].median()}   Mittelwert: {df['B'].mean():.2f}   StdAbw: {df['B'].std():.2f}")
  print("Schwestern: ")
  print(f" Median: {df['S'].median()}   Mittelwert: {df['S'].mean():.2f}   StdAbw: {df['S'].std():.2f}")
  print("Geschwister gesamt: ")
  print(f" Median: {df['G'].median()}   Mittelwert: {df['G'].mean():.2f}   StdAbw: {df['G'].std():.2f}")
  df['text_truncated'] = df['Text'].str[:50]
  top_drei_df = df['text_truncated'].value_counts().head(3).reset_index()
  top_drei_df.columns = ['Text', 'Count']
  print("Häufigste drei Antworten: ")
  for index, row in top_drei_df.iterrows():
      print(f"{row['Text']}   {row['Count'] / num_samples * 100:.1f}%")
  print("Done!")

def update_params(change):
    global temperatur
    global max_tokens
    global model
    global stoptokens
    global raetsel_prompt
    global num_samples
    global token_limit_reached
    # Token-Obergrenze umrechnen
    try:
        max_tokens = int(textbox_max_tokens.value)
        if max_tokens == 0:
            max_tokens = None
    except ValueError:
        max_tokens = None
    textbox_max_tokens.value = f'{max_tokens}'
    raetsel_prompt = area_raetsel.value
    model = dropdown_model.value
    if (stoptokens == ""):
      stoptokens = None
    temperatur = slider_temperatur.value
    num_samples = slider_samples.value
    sample_tokens = calculate_tokens(raetsel_prompt) * num_samples + calculate_tokens(beurteilung_prompt) * num_samples
    html_samples_tokens.value = f'Overhead: {sample_tokens} entsprechend $ {input_pricing(sample_tokens):.2f}'

# Verbinde die Widgets mit der Funktion zur Verarbeitung der Werte
slider_temperatur.observe(update_params, 'value')
slider_samples.observe(update_params, 'value')
dropdown_model.observe(update_params, 'value')
area_raetsel.observe(update_params, 'value')

button_start_experiment.on_click(my_main)

# Einmal alles berechnen...
update_params(0)

# ...dann Einstell-Widgets anzeigen
display(slider_temperatur,
        slider_samples,
        dropdown_model,
        textbox_max_tokens,
        area_raetsel,
        html_samples_tokens,
        button_start_experiment,
        )


Widgets eingerichtet.


FloatSlider(value=1.0, description='Temperatur:', max=2.0)

IntSlider(value=100, description='Durchgänge:', layout=Layout(width='50%'), max=500, min=10, step=10)

Dropdown(description='Modell:', options=('gpt-3.5-turbo-1106', 'gpt-4-1106-preview', 'gpt-3.5-turbo'), value='…

Text(value='256', description='Max. Antwort:', placeholder='0')

...


Richtig beantwortet: 24.2%
Brüder: 
 Median: 3.0   Mittelwert: 3.05   StdAbw: 0.45
Schwestern: 
 Median: 2.0   Mittelwert: 2.32   StdAbw: 1.44
Geschwister gesamt: 
 Median: 5.0   Mittelwert: 5.36   StdAbw: 1.60
Häufigste drei Antworten: 
Das ist ein klassisches Rätsel, das auf den ersten   5.6%
Das ist ein klassisches Rätsel mit einer kleinen F   5.2%
Anna hat drei Brüder, und da jeder Bruder zwei Sch   5.0%
Done!


HTML(value='<b>Verbrauchte Token:</b> 201997 ($ 0.239) ')

Sammele 500 Antworten


HTML(value='\n<div class="loader"></div>\n<style>\n.loader {\n  border: 8px solid #f3f3f3;\n  border-top: 8px …

***********************************************************************************

## Was die Einstellwerte bedeuten:

* **Temperatur:** Wie stark soll der Zufall das Sprachmodell beeinflussen? (0 = wenig, 1 = Normaleinstellung, 1,5 = Irrsinnsgrenze, ab der das Modell delirisch wird)
* **Durchgänge:** Wie oft soll die Rätselfrage dem Modell gestellt und von ihm neu beantwortet werden?
* **Modell:** Welches Sprachmodell soll befragt werden? Standard ist ```GPT-3.5-turbo-1106```, das (im Jan. 2024) neuesten Update des Sprachmodells, das auch ChatGPT antreibt. Außerdem steht die neue Variante des mächtigeren Modells ```GPT-4``` zur Auswahl - wenn der API-Key es zulässt - und zum Vergleich eine GPT3.5-Variante aus dem Juni 2023, ```gpt-3.5-turbo-0613```
* **Max. Antwort:** Wie lang darf die Antwort des Sprachmodells werden? Gerade, wenn der Zufall das Modell zum Schwafeln bringt, ist diese Begrenzung nötig - sonst hört es nicht mehr auf. Die Standardeinstellung von 256 Tokens - entspricht im Deutschen etwa 180 Wörtern - reicht für eine korrekte Antwort dicke aus. Wer keine Längenbegrenzung haben will, stellt 0 ein.
* **Rätselfrage:** Das ist der Prompt, den das Sprachmodell beantworten muss. Es bekommt kein Systemprompt dazu, ist also in der Standardrolle eines bemühten und hilfreichen Assistenten.

Der **Overhead** gibt an, wie viele Tokens erzeugt werden, um dem Sprachmodell die Frage so oft wie vorgegeben stellen zu können - und die Antworten zu bewerten. Er erlaubt eine erste Abschätzung der Kosten - allerdings kommt noch ein Mehrfaches des Overheads für die erzeugten Antworten hinzu (die noch nicht berechnet werden können).

Sobald das Experiment gestartet ist, rechnet das Programm die laufenden Kosten auf. Sie sind für GPT-4 10x so hoch wie für das weniger mächtige Sprachmodell GPT-3.5